## Overview
---
Consider $N$ molecules diffusing in the one-dimensional interval $[-L, L]$ with reflective boundary conditions. A molecular-based model of a similar process was presented as the $\mathrm{SSA}$ in notebook (4.1). Denoting the positions of molecules as $X_{i}(t), i=1,2, \ldots, N, t \geq 0,$ the position of the $i$ th molecule at time $t+\Delta t$ can be computed using:

$$
X_{i}(t+\Delta t)=X_{i}(t)+\sqrt{2 D \Delta t} \xi_{i}\text{---(1)}
$$

>Our goal is to **present multi-resolution methods which use a compartment-based model in some parts of the computational domain**. 

We use a simple test problem shown below:
![Schematic of the computational domain](9-1.PNG)

We divide the computational domain $[-L, L]$ into two parts $\Omega_{\mathrm{c}}=[-L, 0)$ and $\Omega_{\mathrm{m}}=(0, L]$ separated by the boundary (interface) $I=\{0\}$. We use the compartment-based model in $\Omega_{c}$ and the molecular-based model in $\Omega_{m} .$ In our case, we divide the compartment-based subdomain $\Omega_{\mathrm{c}}$ into $K$ compartments of length $h=L / K .$ We denote the number of molecules in the $i$ th compartment $[-i h,-(i-1) h)$ by $A_{i}, i=1,2, \ldots, K .$ Then the compartment-based model is given as the following chain of "chemical reactions":

$$
A_K\substack{\overset{d}{\rightarrow}\\ \underset{d}{\leftarrow}}A_{K-1}\dots\substack{\overset{d}{\rightarrow}\\ \underset{d}{\leftarrow}}A_1\text{---(2)}
$$

## Reformulate Transition of Molecules between $\Omega_{\mathrm{c}}$ and $\Omega_{\mathrm{m}}$
---
In this case we use a time-driven algorithm instead of the Gillespie SSA. **Then diffusive jumps in (2) can be simulated using the same finite time step $\Delta t$ as in (1).** A diffusive jump to the right/left from the ith compartment, $i = 2, 3, . . . , K$, occurs with probability:
$$
d A_{i}(t) \Delta t=\frac{D}{h^{2}} A_{i}(t) \Delta t
$$

Next, we formulate the transition of molecules between $\Omega_{c}$ and $\Omega_{m}$ with the help of two parameters: a positive constant $\Phi,$ and a probability distribution function $f:[0, L] \rightarrow[0, \infty),$ where $\int_{0}^{L} f(x) \mathrm{d} x=1 .$ Possible choices of $\Phi$ and $f(x)$ are discussed here.

>We postulate that **the transition rate for moving across the interface $I$ from $\Omega_{\mathrm{c}}$ to $\Omega_{\mathrm{m}}$ is $\Phi d,$ i.e. it is equal to the internal transition rate $d=D / h^{2}$ multiplied by the constant $\Phi .$** In particular, jumps from the first compartment to the right can also be written as the following first-order chemical reaction:

>$A_{1} \overset{\Phi d}{\longrightarrow}$ transition to $\Omega_{\mathrm{m}} .$

If a molecule jumps from the first compartment to the right, then a new molecule is initiated in $\Omega_{\mathrm{m}}$ at a position which is sampled according to the distribution $f(x) .$ On the other hand, if a molecule (which was in $\Omega_{\mathrm{m}}$ at time $t$ ) crosses the interface during the time step $[t, t+\Delta t),$ then it will be incorporated
into the first compartment in $\Omega_{\mathrm{c}} .$

## Multi-resolution SSA
---
>1. Set $\delta A_{i}=0$ for $i=1,2, \ldots, K$

>2. Generate $K-1$ random numbers $r_{i}^{L}, i=1,2, \ldots, K-1,$ uniformly distributed in $(0,1) .$ **If $r_{i}^{L}<d A_{i}(t) \Delta t,$ then $\delta A_{i}=\delta A_{i}-1$ and $\delta A_{i+1}=\delta A_{i+1}+1$**

>3. Generate $K-1$ random numbers $r_{i}^{R}, i=2,3, \ldots, K,$ uniformly distributed in $(0,1) .$ **If $r_{i}^{R}<d A_{i}(t) \Delta t,$ then $\delta A_{i}=\delta A_{i}-1$ and $\delta A_{i-1}=\delta A_{i-1}+1$**

>4. Compute the position of each molecule (which is in
molecular-based subdomain $\Omega_{m}$ ) at time $t+\Delta t,$ i.e. compute $X_{i}(t+\Delta t)$ using (1) for $i=1,2, \ldots, N_{\mathrm{m}}(t) .$

>5. If $X_{i}(t+\Delta t), i=1,2, \ldots, N_{\mathrm{m}}(t),$ computed by (1) is greater than $L,$ then use the reflective boundary condition
$$
X_{i}(t+\Delta t)=2 L-X_{i}(t)-\sqrt{2 D \Delta t} \xi_{i}
$$

>6. If $X_{i}(t+\Delta t), i=1,2, \ldots, N_{\mathrm{m}}(t),$ computed by (1) is less than $0,$
then $\delta A_{1}=\delta A_{1}+1$ and the trajectory of the $i$ th molecule is terminated.

>7. If $X_{i}(t+\Delta t), i=1,2, \ldots, N_{\mathrm{m}}(t),$ computed by ( 9.1) is greater than $0,$ then generate a random number $r_{1}$ uniformly distributed in $(0,1) .$ If $r_{1}<\exp \left[-X_{i}(t+\Delta t) X_{i}(t) /(D \Delta t)\right],$ then $\delta A_{1}=\delta A_{1}+1$ and the trajectory of the $i$ th molecule is terminated.

>8. Generate a random number $r_{2}$ uniformly distributed in $(0,1) .$ If $r_{2}<\Phi d A_{1}(t) \Delta t,$ then $\delta A_{1}=\delta A_{1}-1$ and a new molecule is introduced at position $x \in \Omega_{\mathrm{m}}$ sampled according to distribution
$f(x)$

>9. Update numbers of molecules in all compartments using $A_{i}(t+\Delta t)=A_{i}(t)+\delta A_{i}, \quad$ for $i=1,2, \ldots, K$

## Specify $\phi$ and $f(x)$
---
Since the jump rate between compartments is equal to the same value, $d,$ for all reactions in $(9.2),$ it might look natural to impose the same transition rate for moving across the interface $I$ from $\Omega_{c}$ to $\Omega_{m},$ which would mean $\Phi=1 .$ When a molecule moves from $\Omega_{c}$ to $\Omega_{m},$ we sample its initial position in $\Omega_{\mathrm{m}}$ according to distribution $f(x)$. One possibility is to assume that the initial position of each new molecule is estimated according to the uniform distribution in $[0, h] .$ Therefore we could simply postulate

$$
\Phi=1 \quad \text { and } \quad f(x) \equiv\left\{\begin{array}{cc}{1 / h} & {\text { for } x \in[0, h]} \\ {0} & {\text { otherwise }}\end{array}\right.\text{---(3)}
$$

### Disproof of the Postulate(3)
---
Run SSA for a long time with the postulate (3), we can compare the result with **analytical steady-state distribution, which is a uniform density given by:**

$$
n_{s}(x)=\frac{N}{2 L}
$$

**We would get large error is caused by the incorrect implementation of the diffusive transport across the artificial interface $I$.**

### Find more Accurate $\Phi$ and $f(x)$

**Let's first consider the SSA for $N=1$ molecule**. We can equivalently describe this problem using mathematical equations, which can be analysed to get the correct values of  $\Phi$ and $f(x)$. To simplify the resulting equations, we replace the finite domain $[-L, L],$ by the infinite domain, $(-\infty, \infty)$. Then we do not need to worry about boundary conditions.

Using $N=1,$ we denote by $p_{i}(t)$ the probability that the considered molecule is in the ith compartment, $i$, 2, $3, \ldots .$ Let $p(x, t)$ be its probability density in $\Omega_{m}=(0, \infty) .$ Using similar arguments as we used to derive the chemical master equation, the random walk in the $i$ th compartment can be described by

$$
p_{i}(t+\Delta t)=\left(1-2 \frac{D \Delta t}{h^{2}}\right) p_{i}(t)+\frac{D \Delta t}{h^{2}}\left(p_{i-1}(t)+p_{i+1}(t)\right), \quad \text { for } i \geq 2
$$

For $p_1$ and $p(x,t)$, we can describe them using evolution equations:

$$
\begin{aligned} p_{1}(t+\Delta t)=&\left(1-\frac{(1+\Phi) D \Delta t}{h^{2}}\right) p_{1}(t)+\frac{D \Delta t}{h^{2}} p_{2}(t) \\ &+\frac{1}{h \sqrt{4 \pi D \Delta t}} \int_{0}^{\infty} \int_{0}^{\infty} p(y, t) \exp \left(-\frac{(x+y)^{2}}{4 D \Delta t}\right) \mathrm{d} x \mathrm{d} y \\ &+\frac{1}{h \sqrt{4 \pi D \Delta t}} \int_{0}^{\infty} \int_{0}^{\infty} p(y, t) \exp \left(-\frac{x y}{D \Delta t}\right) \exp \left(-\frac{(x-y)^{2}}{4 D \Delta t}\right) \mathrm{d} x \mathrm{d} y \end{aligned}\text{---(4)}
$$

The third term at RHS represents the possibility that none of the particles in $\Omega_m$ passed the interface $I$ during $\Delta t$. The fourth term represents the possibility of particles in $\Omega_m$ passing $I$ and then back to $\Omega_m$ during $\Delta t$. Please refer to the notebook (4.4) for the details of proof. Similarly, $p(x,t+\Delta t)$ can be derived as:

$$
\begin{aligned} p(x, t+\Delta t)=& \frac{1}{\sqrt{4 \pi D \Delta t}} \int_{0}^{\infty} p(y, t)\left[\exp \left(-\frac{(x-y)^{2}}{4 D \Delta t}\right)-\exp \left(-\frac{(x+y)^{2}}{4 D \Delta t}\right)\right] \mathrm{d} y \\ &+\frac{D \Delta t \Phi}{h} f(x) p_{1}(t) \end{aligned}\text{---(5)}
$$

[Flegg et al. ( 2012)](https://royalsocietypublishing.org/doi/full/10.1098/rsif.2011.0574) analysed equations $(4)-(5)$ using the method of matched asymptotic expansions with the aim of finding the values of $\Phi$ and $f(x)$ that minimize the error at interface $I=\{0\} .$ Such an analysis leads to the following formulae for $\Phi$ and $f(x):$

$$
\Phi=\frac{2 h}{\sqrt{\pi D \Delta t}}, \quad f(x)=\frac{\pi}{4 D \Delta t} \operatorname{erfc}\left(\frac{x}{\sqrt{4 D \Delta t}}\right), \quad x \in \Omega_{\mathrm{m}}\text{--(6)}
$$

where

$$
\operatorname{erfc}(z)=\frac{2}{\sqrt{\pi}} \int_{z}^{\infty} \exp \left(-y^{2}\right) \mathrm{d} y
$$

One advantage of formula ($6$) is that **it can be applied in simulations even by modellers who might not be interested in its mathematical derivation. All
they need is a method to sample random numbers from probability distribution $f(x)$**.